This notebook plots the scores of specific submissions over time to help you understand how long it takes for your agent to reach its approximate skill level 
and maybe to let us estimate value from some early episodes.

I took much guidance from @robga's excellent episode scraper downloader.  We don't need the actual episodes here, just the scores from 
the dataset.

It uses the [Meta Kaggle](https://www.kaggle.com/kaggle/meta-kaggle) dataset.

To run this notebook you might need to re-add the Meta Kaggle dataset. After opening your copy of the notebook, click "+ Add data" top right in the notebook editor.

NOTE: I found that I did not need to do this after copying the notebook, but if you can't see the data-kaggle directory under Input, you will need to do this.


Meta Kaggle is refreshed daily, but sometimes misses daily refreshes for a few days.


In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime
import time
import glob
import collections

In [ ]:
## You should configure these to your needs. Choose one of ...
# 'hungry-geese', 'rock-paper-scissors', santa-2020', 'halite', 'google-football'
COMP = 'kore-2022'
MAX_CALLS_PER_DAY = 300 # Kaggle says don't do more than 3600 per day and 1 per second

In [ ]:
ROOT ="../working/"
META = "../input/meta-kaggle/"
MATCH_DIR = '../working/'
#base_url = "https://www.kaggle.com/requests/EpisodeService/"
base_url = "https://www.kaggle.com/api/i/competitions.EpisodeService/"
    
get_url = base_url + "GetEpisodeReplay"
BUFFER = 1
COMPETITIONS = {
    'kore-2022': 34419,
    'lux-ai-2021': 30067,
    'hungry-geese': 25401,
    'rock-paper-scissors': 22838,
    'santa-2020': 24539,
    'halite': 18011,
    'google-football': 21723
}
#SubmissionIds I care about.  You can get these from your My Submissions take and click 'View'
#TODO: You could get them by TeamId by using another data file
#Put your submission IDs that you care about here
myIds=[27124743,27170784,27138146,27165648,27163973]

In [ ]:
# Filter Episodes.csv
data = pd.read_csv(META + "Episodes.csv", chunksize=1e6)
df_list = [] 
for chunk in data:
    df_list.append(chunk[chunk['CompetitionId']==COMPETITIONS[COMP]])
episodes_df = pd.concat(df_list)
del data
del chunk
del df_list
print(f'Episodes.csv: {len(episodes_df)} rows after filtering for {COMP}.')

In [ ]:
# Filter EpisodeAgents.csv
#This has the Agents (Submissions) and their scores.  This is all we will need if we do not care about times of episodes 
#which we could get from Episodes.csv
data = pd.read_csv(META + "EpisodeAgents.csv", chunksize=1e6)
df_list = [] 
for i, chunk in enumerate(data):
    df_list.append(chunk[chunk.SubmissionId.isin(myIds)])
    if i%5==0:
        print(f'{i} {len(epagents_df)}')
epagents_df = pd.concat(df_list)
del data
del chunk
del df_list
print(f'EpisodeAgents.csv: {len(epagents_df)} rows after filtering for {myIds}.')

In [ ]:
#Let's look at one agent
pd.set_option('display.max_rows', 20)
agent1 = epagents_df[epagents_df['SubmissionId']==myIds[1]]
agent1


Some things to note here.  They seem to be in Id order, but that is not time order of the episodes, which we can tell from the
InitialScore and UpdatedScore columns.  They should connect, UpdatedScore[i] should match InitialScore[i+1] when they are in
the correct order.  We know the initial score should be 600 for the first real episode and the UpdatedScore should be 600 for that first test episode where we play ourself.

In [ ]:
agent1.loc[(agent1['InitialScore']==600) | (agent1['UpdatedScore']==600)]

In [ ]:
#Let's check them in EpisodeId order
agent1=agent1.sort_values('EpisodeId')
pd.set_option('display.max_rows', 200)
agent1

This looks good.  I suppose it is possible that EpisodeId is not the exact order things are scored in if, for example, the episodes are carried out in a multi-processing environment, and episodes could be finished out of order.  However, this looks good enough for our purposes, as we just want to plot a score 
trajectory

In [ ]:
import matplotlib.pyplot as plt
agent1.reset_index(inplace=True, drop=True)
agent1.plot(y='UpdatedScore',kind='line')


In [ ]:
#Now we can plot all the agents
fig, ax = plt.subplots()

#for key, grp in df.groupby(['color']):
#    ax = grp.plot(ax=ax, kind='line', x='x', y='y', c=key, label=key)

for x in myIds:
    agent = epagents_df[epagents_df['SubmissionId']==x]
    agent = agent.sort_values('EpisodeId')
    agent.reset_index(inplace=True, drop=True)
    agent.plot(y='UpdatedScore', kind='line',ax=ax, label=x)
#plt.show()

It looks like it takes about 50 episodes to reach a reasonable steady state value (of course, it then trends down over time as opponents get better!).  This generally happens within the first 24 hours or so.

Happy Mining!